<a href="https://colab.research.google.com/github/binnu02/CODSOFT/blob/main/TASK3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sklearn


In [2]:
pip install scikit-surprise


In [3]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate

# Sample data of user-item ratings
data = {
    'user_id': ['User1', 'User1', 'User2', 'User2', 'User3', 'User3', 'User4', 'User4'],
    'item_id': ['Movie1', 'Movie2', 'Movie2', 'Movie3', 'Movie1', 'Movie3', 'Movie4', 'Movie5'],
    'rating': [5, 4, 3, 5, 2, 4, 3, 1]
}

ratings_df = pd.DataFrame(data)

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['user_id', 'item_id', 'rating']], reader)
trainset = data.build_full_trainset()

# Define the similarity measure (cosine similarity)
sim_options = {
    'name': 'cosine',
    'user_based': True
}

# Create a KNN model with collaborative filtering
model = KNNBasic(sim_options=sim_options)

# Train the model on the training set
model.fit(trainset)

# Function to get movie recommendations for a user
def get_movie_recommendations(user_id, n=5):
    try:
        # Get a list of all movie IDs
        all_movie_ids = list(trainset.all_items())

        # Remove movies the user has already seen
        movies_seen_by_user = [item[0] for item in trainset.ur[trainset.to_inner_uid(user_id)]]
        movies_to_predict = [movie_id for movie_id in all_movie_ids if movie_id not in movies_seen_by_user]

        if not movies_to_predict:
            raise ValueError("No movies left to predict. User has interacted with all available items.")

        # Predict ratings for movies not seen by the user
        predictions = [model.predict(trainset.to_inner_uid(user_id), movie_id) for movie_id in movies_to_predict]

        # Sort the predictions by estimated rating in descending order
        sorted_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)

        # Get the top N recommendations
        top_n_recommendations = sorted_predictions[:n]

        # Extract movie indices from their predictions
        movie_indices = [item[0] for item in top_n_recommendations]

        # Get movie titles from the original movies dataframe
        recommended_movies = trainset.to_inner_iid
        recommended_movie_titles = [recommended_movies(movie_id) for movie_id in movie_indices]

        return recommended_movie_titles

    except ValueError as e:
        print(f"Error: {e}")
        return []

# Example usage
user_id = 'User1'  # Replace with the ID of the user for whom you want to get recommendations
recommendations = get_movie_recommendations(user_id, n=5)

print(f"Recommended movies for User {user_id}:")
print(recommendations)



Computing the cosine similarity matrix...
Done computing similarity matrix.
Error: Item 0 is not part of the trainset.
Recommended movies for User User1:
[]
